<a href="https://colab.research.google.com/github/SAK90/TradingStrategies/blob/master/Equal_Weight_S%26P500_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import requests  
!pip install xlsxwriter
import xlsxwriter
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 7.7 MB/s 


In [5]:
from google.colab import files
uploaded = files.upload()

Saving sp_500_stocks.csv to sp_500_stocks.csv


In [7]:
import io
stocks = pd.read_csv(io.BytesIO(uploaded['sp_500_stocks.csv']))
# there are 505 columns which means that some stocks with dual share structure
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [8]:
# public IEX sandbox token so not hidden
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [26]:
# Loop over all the listed stocks in the S&P and make a get request for each stock and find it's stock price and market capitalization (number of stock * price of a stock)
my_columns = ['Ticker', 'Stock price', 'Market capitalization', 'Number of shares to buy']
output_dataframe = pd.DataFrame(columns = my_columns)
index = 0
for stock_name in stocks['Ticker']:
  api_url = f'https://sandbox.iexapis.com/stable/stock/{stock_name}/quote?token={IEX_CLOUD_API_TOKEN}'
  data = requests.get(api_url)
  if data.status_code == 404:
    stocks.drop([index], inplace=True)
  index+=1
stocks


,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [35]:
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Stock price,Market capitalization,Number of shares to buy
0,A,158.800,45445870777,N/A
1,AAL,14.710,9258707885,N/A
2,AAP,150.700,9130918854,N/A
3,AAPL,146.700,2307684444117,N/A
4,ABBV,164.663,281037788338,N/A
...,...,...,...,...
496,YUM,130.740,36729389985,N/A
497,ZBH,119.940,25150252521,N/A
498,ZBRA,272.230,14118685017,N/A
499,ZION,52.458,7683713178,N/A


In [28]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:1000000


In [41]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number of shares to buy'] = math.floor(position_size / final_dataframe['Stock price'][i])
final_dataframe

,Ticker,Stock price,Market capitalization,Number of shares to buy
0,A,158.800,45445870777,12
1,AAL,14.710,9258707885,135
2,AAP,150.700,9130918854,13
3,AAPL,146.700,2307684444117,13
4,ABBV,164.663,281037788338,12
...,...,...,...,...
496,YUM,130.740,36729389985,15
497,ZBH,119.940,25150252521,16
498,ZBRA,272.230,14118685017,7
499,ZION,52.458,7683713178,38


In [42]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [43]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [44]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Stock price', dollar_format],
                    'C': ['Market capitalization', dollar_format],
                    'D': ['Number of shares to buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [45]:
writer.save()

In [46]:
files.download('recommended_trades.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>